In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import os
os.chdir('/home/arya/Desktop/handson_ml_study/')

In [4]:
#getting the data
housing_data = pd.read_csv('./handson-ml2-master/datasets/housing/housing.csv')
housing_data = housing_data
housing_data["income_cat"] = pd.cut(housing_data["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])

In [5]:
#stratified classified train test split
from sklearn.model_selection import StratifiedShuffleSplit

strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=32)
for train_index, test_index in strat_split.split(housing_data, housing_data["income_cat"]):
        strat_train_set = housing_data.loc[train_index]
        strat_test_set = housing_data.loc[test_index]


for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [6]:
strat_train_set

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
5242,-118.40,34.11,32.0,5578.0,753.0,1567.0,697.0,15.0001,500001.0,<1H OCEAN
18762,-122.25,40.66,15.0,2771.0,546.0,1423.0,505.0,3.6413,108500.0,INLAND
10729,-117.87,33.60,20.0,3212.0,572.0,1064.0,526.0,6.6155,500001.0,<1H OCEAN
17547,-121.88,37.34,52.0,867.0,232.0,1264.0,227.0,2.6312,302900.0,<1H OCEAN
3337,-122.51,38.76,9.0,2589.0,482.0,1050.0,374.0,4.0435,132600.0,INLAND
...,...,...,...,...,...,...,...,...,...,...
5043,-118.33,33.98,30.0,3112.0,931.0,2739.0,841.0,1.6531,118500.0,<1H OCEAN
7931,-118.06,33.84,26.0,6960.0,1454.0,4367.0,1437.0,4.7953,210900.0,<1H OCEAN
7763,-118.10,33.91,36.0,726.0,NaN,490.0,130.0,3.6389,167600.0,<1H OCEAN
19433,-121.04,37.65,8.0,1959.0,379.0,995.0,365.0,3.3567,129100.0,INLAND


In [7]:
#studying correlations between attributes and their combinations
housing_explore = strat_train_set.copy()
corr_matrix = housing_explore.corr()
housing_explore["rooms_per_household"] = housing_explore["total_rooms"]/housing_explore["households"]
housing_explore["bedrooms_per_room"] = housing_explore["total_bedrooms"]/housing_explore["total_rooms"]
housing_explore["population_per_household"]=housing_explore["population"]/housing_explore["households"]
corr_matrix = housing_explore.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

#this is just exploring these didn't ge

median_house_value          1.000000
median_income               0.684216
rooms_per_household         0.166727
total_rooms                 0.133887
housing_median_age          0.111445
households                  0.066193
total_bedrooms              0.050271
population                 -0.026870
population_per_household   -0.033058
longitude                  -0.043955
latitude                   -0.145869
bedrooms_per_room          -0.250116
Name: median_house_value, dtype: float64

In [8]:
#encoding categorical to numerical
# from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder()

ocean_proxi = one_hot.fit_transform(housing_explore[['ocean_proximity']])


In [9]:
#fixn gmissin gvalues
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

housing_explore_labels = housing_explore['median_house_value'].copy()
housing_explore = housing_explore.drop('median_house_value', axis = 1)
housing_num = housing_explore.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)
X = imputer.transform(housing_num)
housing_explore = pd.DataFrame(X, columns=housing_num.columns)

In [10]:
one_hot.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [11]:
housing_explore

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
0,-118.40,34.11,32.0,5578.0,753.0,1567.0,697.0,15.0001,8.002869,0.134995,2.248207
1,-122.25,40.66,15.0,2771.0,546.0,1423.0,505.0,3.6413,5.487129,0.197041,2.817822
2,-117.87,33.60,20.0,3212.0,572.0,1064.0,526.0,6.6155,6.106464,0.178082,2.022814
3,-121.88,37.34,52.0,867.0,232.0,1264.0,227.0,2.6312,3.819383,0.267589,5.568282
4,-122.51,38.76,9.0,2589.0,482.0,1050.0,374.0,4.0435,6.922460,0.186172,2.807487
...,...,...,...,...,...,...,...,...,...,...,...
16507,-118.33,33.98,30.0,3112.0,931.0,2739.0,841.0,1.6531,3.700357,0.299165,3.256837
16508,-118.06,33.84,26.0,6960.0,1454.0,4367.0,1437.0,4.7953,4.843424,0.208908,3.038970
16509,-118.10,33.91,36.0,726.0,436.0,490.0,130.0,3.6389,5.584615,0.203028,3.769231
16510,-121.04,37.65,8.0,1959.0,379.0,995.0,365.0,3.3567,5.367123,0.193466,2.726027


In [12]:
#custom transformers
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 4,5,6,7

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
         # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room


    def fit(self, X, y=None):
        return self # nothing else to do def transform(self, X, y=None):


    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix] 
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, bedrooms_per_room, population_per_household]
            
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [13]:

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing_explore.values)

In [14]:
cv = pd.DataFrame(housing_extra_attribs)
cv

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-118.40,34.11,32.0,5578.0,753.0,1567.0,697.0,15.0001,8.002869,0.134995,2.248207,50.199665,46.466357
1,-122.25,40.66,15.0,2771.0,546.0,1423.0,505.0,3.6413,5.487129,0.197041,2.817822,149.946448,138.686733
2,-117.87,33.60,20.0,3212.0,572.0,1064.0,526.0,6.6155,6.106464,0.178082,2.022814,86.463608,79.510241
3,-121.88,37.34,52.0,867.0,232.0,1264.0,227.0,2.6312,3.819383,0.267589,5.568282,88.172697,86.272423
4,-122.51,38.76,9.0,2589.0,482.0,1050.0,374.0,4.0435,6.922460,0.186172,2.807487,119.203660,92.494126
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16507,-118.33,33.98,30.0,3112.0,931.0,2739.0,841.0,1.6531,3.700357,0.299165,3.256837,563.184320,508.741153
16508,-118.06,33.84,26.0,6960.0,1454.0,4367.0,1437.0,4.7953,4.843424,0.208908,3.038970,303.213563,299.668425
16509,-118.10,33.91,36.0,726.0,436.0,490.0,130.0,3.6389,5.584615,0.203028,3.769231,119.816428,35.725082
16510,-121.04,37.65,8.0,1959.0,379.0,995.0,365.0,3.3567,5.367123,0.193466,2.726027,112.908511,108.737748


In [15]:
#transformation pipelines
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])


In [16]:
#from here go with my_pipeline_pd

from sklearn.compose import ColumnTransformer

num_attri = list(strat_train_set.copy().drop(["ocean_proximity","median_house_value"], axis=1))
cat_attri = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attri),
    ('oneHot', OneHotEncoder(), cat_attri)
    
])
housing_prepared = full_pipeline.fit_transform(strat_train_set.copy())
housing_prepared = pd.DataFrame(housing_prepared)
housing_prepared

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.587172,-0.716065,0.261033,1.350108,0.514600,0.120138,0.515879,5.831124,-0.771510,-0.146138,-0.790025,1.0,0.0,0.0,0.0,0.0
1,-1.330864,2.343295,-1.091440,0.059972,0.019956,-0.006017,0.011601,-0.121474,-0.127440,-0.047872,-0.128850,0.0,1.0,0.0,0.0,0.0
2,0.851214,-0.954275,-0.693654,0.262661,0.082086,-0.320527,0.066757,1.437160,-0.537352,-0.187461,-0.553116,1.0,0.0,0.0,0.0,0.0
3,-1.146534,0.792597,1.852177,-0.815133,-0.730372,-0.145312,-0.718551,-0.650819,-0.526316,0.483861,-0.504635,1.0,0.0,0.0,0.0,0.0
4,-1.460394,1.455847,-1.568783,-0.023678,-0.132977,-0.332792,-0.332463,0.089299,-0.325948,-0.127912,-0.460028,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16507,0.622046,-0.776785,0.101918,0.216700,0.939945,1.146897,0.894087,-1.163394,2.540858,0.014949,2.524260,1.0,0.0,0.0,0.0,0.0
16508,0.756558,-0.842176,-0.216311,1.985294,2.189696,2.573146,2.459448,0.483281,0.862214,0.026444,1.025310,1.0,0.0,0.0,0.0,0.0
16509,0.736630,-0.809481,0.579262,-0.879939,-0.242897,-0.823394,-0.973316,-0.122732,-0.321991,-0.325217,-0.867034,1.0,0.0,0.0,0.0,0.0
16510,-0.728053,0.937391,-1.648340,-0.313235,-0.379103,-0.380976,-0.356101,-0.270620,-0.366596,-0.044298,-0.343569,0.0,1.0,0.0,0.0,0.0


In [17]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(housing_prepared, housing_explore_labels)

LinearRegression()

In [18]:
some_data = housing_data.iloc[:5]
some_labels = housing_data['median_house_value'][:5]
some_data = full_pipeline.transform(some_data)
model.predict(some_data)

array([408743.68224748, 423165.68257215, 379189.80498521, 321666.44373466,
       257114.1299293 ])

In [19]:
some_labels

0    452600.0
1    358500.0
2    352100.0
3    341300.0
4    342200.0
Name: median_house_value, dtype: float64

In [20]:
from sklearn.metrics import mean_squared_error
predictions = model.predict(some_data)
mse = mean_squared_error(some_labels, predictions)
rmse = np.sqrt(mse)
rmse

53784.69384644672

In [21]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor()
tree.fit(housing_prepared, housing_explore_labels)
tree_predict = tree.predict(some_data)

In [22]:
tree_predict

array([452600., 358500., 352100., 341300., 342200.])

In [23]:
mse = mean_squared_error(some_labels, tree_predict)
rmse = np.sqrt(mse)
rmse

0.0

In [24]:
#k fold cross validation on model
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree, housing_prepared, housing_explore_labels, scoring='neg_mean_squared_error', cv=10)


In [25]:
tree_rmse_scores = np.sqrt(-scores)

In [26]:
print(tree_rmse_scores.mean(), tree_rmse_scores.std())

71065.6803905592 2446.147279378443


In [27]:
linear_cross_valid_scores = cross_val_score(model, housing_prepared, housing_explore_labels, scoring='neg_mean_squared_error', cv=10)

In [28]:
lin_rmse_scores = np.sqrt(-linear_cross_valid_scores)

In [29]:
print(lin_rmse_scores.mean(), lin_rmse_scores.std())

69319.11759297753 1914.2031593678694


In [30]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=100, random_state=42)
forest.fit(housing_prepared, housing_explore_labels)

RandomForestRegressor(random_state=42)

In [31]:
housing_predictions = forest.predict(housing_prepared)
forest_mse = mean_squared_error(housing_explore_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

19505.010044383467

In [32]:
forest_scores = cross_val_score(forest, housing_prepared, housing_explore_labels, scoring="neg_mean_squared_error", cv=10)
forest_scores = np.sqrt(-forest_scores)
print(forest_scores.mean(), forest_scores.std()) 

50386.62031932768 1717.923248992611


In [33]:
import joblib

In [34]:
#joblib.dump(model, 'my_chap2_lin_model.pkl')

In [35]:
#joblib.dump(tree, 'my_chap2_dec_tree_model.pkl')
#joblib.dump(forest, 'my_chap2_forest_model.pkl')

In [36]:
# next time just run all cells 
from sklearn.model_selection import GridSearchCV

In [37]:
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()
gris_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
gris_search.fit(housing_prepared, housing_explore_labels)


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [38]:
gris_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [39]:
gris_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30)

In [40]:
cv_scores = gris_search.cv_results_
for mean_score, params in zip(cv_scores['mean_test_score'], cv_scores['params']):
    print(np.sqrt(-mean_score), params)

64027.38022279006 {'max_features': 2, 'n_estimators': 3}
56025.34213497826 {'max_features': 2, 'n_estimators': 10}
53507.571527502565 {'max_features': 2, 'n_estimators': 30}
61540.930181158685 {'max_features': 4, 'n_estimators': 3}
54421.1099328529 {'max_features': 4, 'n_estimators': 10}
51770.868737265504 {'max_features': 4, 'n_estimators': 30}
59443.63799940489 {'max_features': 6, 'n_estimators': 3}
52897.26485315661 {'max_features': 6, 'n_estimators': 10}
51029.1253309248 {'max_features': 6, 'n_estimators': 30}
59568.59749089529 {'max_features': 8, 'n_estimators': 3}
52838.230178731515 {'max_features': 8, 'n_estimators': 10}
50992.940135154044 {'max_features': 8, 'n_estimators': 30}
62259.29846855353 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
55503.50139381938 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
61940.23739570317 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
54572.712922141545 {'bootstrap': False, 'max_features': 3, 'n_estimators'

In [41]:
gris_search.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_features', 'param_n_estimators', 'param_bootstrap', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score'])

In [42]:
feature_importances = gris_search.best_estimator_.feature_importances_
feature_importances

array([0.07491305, 0.06973191, 0.04974665, 0.02867117, 0.01785247,
       0.015939  , 0.01656567, 0.39864968, 0.03774806, 0.10726398,
       0.02659174, 0.00591061, 0.14467837, 0.        , 0.00189281,
       0.00384484])

In [43]:
full_pipeline.named_transformers_

{'num': Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                 ('attribs_adder', CombinedAttributesAdder()),
                 ('std_scaler', StandardScaler())]),
 'oneHot': OneHotEncoder(),
 'remainder': 'drop'}

In [44]:
extra_attribs = ['rooms_per_hhold', 'popul_pe_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['oneHot']

In [45]:
cat_one_hot_attribs = cat_encoder.categories_[0]
attributes = num_attri + extra_attribs + list(cat_one_hot_attribs)

In [46]:
attributes

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_hhold',
 'popul_pe_hhold',
 'bedrooms_per_room',
 '<1H OCEAN',
 'INLAND',
 'ISLAND',
 'NEAR BAY',
 'NEAR OCEAN']

In [47]:
final_model = gris_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

In [48]:
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

49378.93841192487

In [49]:
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

array([47313.76075297, 51361.14446533])

In [99]:
from sklearn.svm import SVR


In [91]:
svr_predictions= svr.predict(housing_prepared)
svr_mse = mean_squared_error(housing_explore_labels, svr_predictions)
svr_rmse = np.sqrt(svr_mse)
svr_rmse

104813.98927148171

In [92]:
svr_scores = cross_val_score(svr, housing_prepared, housing_explore_labels, scoring="neg_mean_squared_error", cv=10)
svr_scores = np.sqrt(-svr_scores)
print(svr_scores.mean(), svr_scores.std())

249938.73396140253 276347.551812026


In [96]:
from sklearn.model_selection import GridSearchCV

param_grid = [
        {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
        {'kernel': ['rbf'], 'C': [300., 1000.0, 1500.0, 2000.0, 3000.0,],
         'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    ]

svm_reg = SVR()
grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(housing_prepared, housing_explore_labels)

Fitting 5 folds for each of 38 candidates, totalling 190 fits
[CV] END ..............................C=10.0, kernel=linear; total time=   0.5s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.5s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.5s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.5s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.5s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.5s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.5s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.6s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.6s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.5s
[CV] END .............................C=100.0, kernel=linear; total time=   0.6s
[CV] END .............................C=100.0, 

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0,
                                10000.0, 30000.0],
                          'kernel': ['linear']},
                         {'C': [300.0, 1000.0, 1500.0, 2000.0, 3000.0],
                          'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
                          'kernel': ['rbf']}],
             scoring='neg_mean_squared_error', verbose=2)

In [97]:
negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

65458.13363529276

In [100]:
grid_search.best_params_

{'C': 3000.0, 'gamma': 0.1, 'kernel': 'rbf'}